# December 2017: Advent of Code Solutions
## Daniel Näslund
From Dec. 1 to Dec. 25, [I](dannas.name) will be solving the puzzles that appear each day at [Advent of Code](http://adventofcode.com/). The two-part puzzles are released at midnight EST (6:00AM CET); points are awarded to the first 100 people to solve the day's puzzles. 

To understand the problems completely, you will have to read the full description in the "[Day 1](http://adventofcode.com/2017/day/1):" link in each day's section header.

##  Prelude
Here I import common functions and modules so I don't have to do it each day.
These are borrowed from [Peter Norvigs Advent of Code solutions](https://github.com/norvig/pytudes/blob/master/ipynb/Advent%20of%20Code.ipynb) from 2016. I've also reused his ipython notebook layout.

In [26]:
# Python 3.x
from itertools import islice, cycle, permutations, count as count_from
from collections import Counter, defaultdict, deque
import re

cat = ''.join
def Input(day):
    filename = 'advent2017/input{}.txt'.format(day);
    return open(filename);
    # TODO(dannas): Fetch the files from elsewhere to allow remote access

def quantify(iterable, pred=bool):
    return sum(map(pred, iterable))
    
# 2-D points implemented using (x, y) tuples
def X(point): return point[0]
def Y(point): return point[1]

def move(pos, direction):
    return (X(pos) + X(direction), Y(pos) + Y(direction))

def neighbors8(point):
    "The eight neighbors (with diagonals)"
    x, y =  point
    return ((x+1, y), (x-1, y), (x, y+1), (x, y-1),
           (x+1, y+1), (x-1, y-1), (x+1, y-1), (x-1, y+1))

def cityblock_distance(p, q=(0, 0)): 
    "City block distance between two points."
    return abs(X(p) - X(q)) + abs(Y(p) - Y(q))


def first(iterable):
    return next(iter(iterable))

def nth(iterable, n, default=None):
    "Returns the nth item or a default value"
    return next(islice(iterable, n, None), default)

def head(iterable, n=10):
    return iterable[:10]

# TODO(dannas): Maybe accept list input as well?
def array(string):
    return [vector(line) for line in string.splitlines()]

def vector(line):
    return [atom(word) for word in line.split()]

def atom(word):
    try:
        return float(word)
    except ValueError:
        try:
            return int(word)
        except ValueError:
            return word

def mapt(fn, iterable):
    return tuple(map(fn, iterable))

def ints(line):
    matches = re.findall(r'\d+', line)
    return map(int, matches)


## [Day 1](http://adventofcode.com/2017/day/1): Inverse Captcha
Given a file of digits, find the sum of all digits that match the next digit in the list. The list is circular, so the digit after the last digit is the first digit in the list.

We need to parse with one token lookahead, it's enough to append the first digit to the end of the list for handling the circular case.

In [ ]:
def pairs(seq):
    return zip(seq[:-1], seq[1:])

def solve_captcha(str):
    digits = [int(d) for d in str] 
    if len(digits) < 2:
        return 0
    digits.append(digits[0])
    return sum(x for x, y in pairs(digits) if x == y)
    
solve_captcha(Input(1).read().strip())

In **part 2** we shall compare the digit halfway around the circular list to the current one. The list has an even number of elements.

This require N/2 token lookahead instad of one. I could have appended the first half of the list to the end, but instead I opted for a circular queue. A circular queue can be implicitely represented using indexes that wrap around; or explicitely using an [ADT](https://docs.python.org/3.6/library/collections.html?highlight=deque#collections.deque.rotate); or using operations on iterators. I choose the later.

The [cycle()](https://docs.python.org/3.6/library/itertools.html#itertools.cycle) function provides an iterator to an infinite circular representation of the list. With [islice](https://docs.python.org/3.6/library/itertools.html#itertools.islice) I can select my start and end position in that list.

In [ ]:
def pairs(seq):
    N = len(seq)
    half = int(N/2)
    x = islice(cycle(seq), 0, N)
    y = islice(cycle(seq), half, N + half)
    return zip(x, y)

def solve_captcha(str):
    digits = [int(d) for d in str] 
    return sum(x for x, y in pairs(digits) if x == y)
    
solve_captcha(Input(1).read().strip())


## [Day 2](http://adventofcode.com/2017/day/2): Corruption Checksum
For each row in a spreadsheet, determine the difference between largest and smallest value; the checksum is the sum of all of these differences.

I initially had some trouble parsing the input into a list of list of integers. It's been a while since I used Python.

In [ ]:
def maxmins(spreadsheet):
    for row in spreadsheet:
        yield max(row), min(row)

def parse(line):
    return tuple(int(x) for x in line.split())

spreadsheet = [parse(line) for line in Input(2)]
sum(x-y for x,y in maxmins(spreadsheet))

In **part 2** we're asked to find the only two numbers in each row where one evenly divides the other - that is, where the result of the division operation is a whole number. Find those numbers on each line, divide them, and add up each line's result.

In [ ]:
def evens(spreadsheet):
   for row in spreadsheet:
        for x, y in permutations(row, 2):
            if x % y == 0:
                yield x, y
                
def parse(line):
    return tuple(int(x) for x in line.split())

spreadsheet = [parse(line) for line in Input(2)]
sum(x/y for x,y in evens(spreadsheet))


## [Day 3](http://adventofcode.com/2017/day/3): Spiral Memory
Walk a a grid in a spiral pattern, a specified number of steps. Then calculate the manhattan distance to the origin.

The trace of steps will form a nested set of cubes. I tried to come up with a neat formula for describing how the the number of points in those cubes increases. Failed. Instead I experimented on paper and found that we increase the sides when we turn East and West.

I decided to represent the points using x,y tuples. I considered using complex numbers first. The walk() function was originally a long list of explicit steps. I then extracted the common parts and ended up with the move and turn functions.

Representing the direction as vectors (2 element tuples of x and y) and using vector addition was a heureka moment for me.

In [5]:
E, N, W, S = [(1, 0), (0, 1), (-1, 0), (0, -1)]    

def move(pos, direction):
    return (X(pos) + X(direction), Y(pos) + Y(direction))

def turn(direction, num_steps):
    changes = {
        E : (N, 0),
        N : (W, 1),
        W : (S, 0),
        S : (E, 1),
    }
    d, delta = changes[direction]
    return d, num_steps + delta


def walk(total_steps):
    pos = (0, 0)
    num_steps = 1
    walked = 0
    direction = E
    
    while total_steps > 0:
        walked += 1
        pos = move(pos, direction)
        if walked == num_steps:
            direction, num_steps = turn(direction, num_steps)
            walked = 0

        total_steps -= 1
    return pos

pos = walk(289326 - 1)

cityblock_distance(pos)

419

In **part 2** we're asked to, for each field, store the sum of previously visited fields, including diagonals. The center point has value 1. Once a field is written, its value doesn't change. 

I spent a lot of time trying to come up with a clever algorithm for determining which neighbors to a field was already visited. Then I had an euphyfany and realized that I could compare all 8 possible adjacent fields, if I had stored a mapping between visited fields and their sums. Determine the first value written that is larger than the puzzle input.

In [ ]:
square_values = {}

def neighbour_sum(pos, direction):
    return sum(square_values[N] for N in neighbors8(pos) if N in square_values)
   
def walk(lower_limit):
    pos = (0, 0)
    square_values[pos] = 1
    num_steps = 1
    walked = 0
    direction = E
    
    while square_values[pos] <= lower_limit:
        walked += 1
        pos = move(pos, direction)
        square_values[pos] = neighbour_sum(pos, direction)
        if walked == num_steps:
            direction, num_steps = turn(direction, num_steps)
            walked = 0
    return square_values[pos]

walk(289326)

## [Day 4](http://adventofcode.com/2017/day/4): High-Entropy Passphrases
How many pass phrases are valid, a.k.a. do not contain duplicated words?

In [ ]:
lines = [L for L in Input(4)]

def is_valid(line):
    words = line.split()
    return len(words) == len(set(words))

sum(1 if is_valid(line) else 0 
    for line in lines)

In **part 2** we're asked how many passphrases are valid, this time with the added requirement that no words in the passphrase can be an anagram of another word.

I remember this one from the chapter Aha! Algorithms in the book Programming Pearls. In that book, Jon meant that detecting anagrams is an example of an algorithm that people may bank their head against the wall with until they finally hits the insight. Thankfully, I was already familiar with it.

In [ ]:
def is_valid(line):
    words = [cat(sorted(w)) for w in line.split()]
    return len(words) == len(set(words))

sum(1 if is_valid(line) else 0
     for line in lines)

## [Day 5](http://adventofcode.com/2017/day/5): A Maze of Twisty Trampolines, All Alike
Given a list of relative jumps, determine the number of steps neccessary to reach a destination outside the list. There's a quirk: The jump instructions increment by one upon being visited.

I first wrote this as a straightforward loop. Had one bug: I interpreted the jumps as absolute positions at first. I'm borrowing the quantify function which I found in Peter Norvigs 2017 Advent of Code solution Jupyter notebook.

In [ ]:
def jumps(instr):
    pos = 0
    while pos >= 0 and pos < N:
        old = pos
        yield pos
        pos += instr[pos]
        instr[old] += 1
    yield pos
        
quantify(jumps([int(x) for x in Input(5)]))

In **part 2** we're asked to again calculate number of steps. After each jump, if the offset was 3 or more, decrement it by 1 else increment it by 1.

I wrote jumps2 as a function that returned the number of steps. Then I rewrote it to yield values instead. That caused the running time to increase from "almost instant" to 8s. Doh. Keeping it as-is, since I accidentily removed the original solution (and I don't understand how  Jupyter handles editing history - I want my vi editor instead of  this webbrowser madness!)

In [ ]:
def jumps2(instr):
    pos = 0
    while pos >= 0 and pos < N:
        old = pos
        yield pos
        offset = instr[pos]
        pos += instr[pos]
        if offset >= 3:
            instr[old] -= 1
        else:
            instr[old] += 1
    yield pos

quantify(jumps2([int(x) for x in Input(5)]))

In [ ]:
%timeit quantify(jumps2([int(x) for x in Input(5)]))

## [Day 6](http://adventofcode.com/2017/day/6): Memory Reallocation
Determine how many cycles it takes before a previously seen permutation of the banks input is seen again.

Let's start with reading the input

In [ ]:
def banks(): return [int(x) for x in Input(6).read().split()]
banks()

The task consists of three parts: find the bank with most blocks, redistribute those blocks, count how many cycles before we're encountering a permutation of banks that has been seen before. I represent each of these tasks with a separate function.

The instructions for restributing the blocks involves a special case for resetting the source bank to zero and fiddly index manipulations. I would have liked to write that part in a closed form list comprehension but it looks clearer this way.

The redistribute part is easier to do inplace with a list than a tuple, but the set requires a tuple as key. Can that be simplified?

In [ ]:
def most_blocks(banks):
    b = max(banks)
    return banks.index(b), b

def redistribute(banks):
    N = len(banks)
    pos, val = most_blocks(banks)
    banks[pos] = 0
    banks[:] = [x + val // N for x in banks]
    for x in range(pos+1, pos+1+ val % N):
        banks[x % N] += 1
    return tuple(banks)

def count_cycles(banks):
    history = {tuple(banks)}
    for cycles in count_from(1):
        result = redistribute(banks)
        if result in history:
            return cycles
        history.add(result)

count_cycles(banks())

In **part 2** we're asked to find the number of cycles from the first duplicated permutation until the next time the same permutation is generated.

I had hoped that I could just reuse the count_cycles() function from part 1, but I didn't manage to accomplish that.

In [ ]:
def count_cycles2(banks):
    history ={tuple(banks)}
    prev = None
    first_found = None
    for cycles in count_from(1):
        result = divide_between(banks)
        if result == prev:
            return cycles - first_found
        if result in history and prev ==  None:
            prev = result
            first_found = cycles
        history.add(result)
count_cycles2(banks())

## [Day 7](http://adventofcode.com/2017/day/7): Recursive Circus
TODO(dannas): Write description

In [ ]:
def parse(line):
    parts = line.split('->')
    parent, weight = parts[0].split()        
    children = [] if len(parts) == 1 else parts[1].split(', ')     
    return (parent, (int(weight[1:-1]), children))
    
tower = dict(parse(line.strip()) for line in Input(7))

roots = {}
for parent in tower:
    for c in tower[parent][1]:
        roots[c] = parent

start = nth(tower, 8)
print(nth(tower, 4))

while start in roots:
    start = roots[start]

print(start)
# TODO(dannas): Write solution for part 1

In **part 2** we're asked to determine which single program has a weight that makes the tower unbalanced.

The tower is naturally represented as a tree; each program can have only parent but many children and they're all rooted in one single base program. 

In [ ]:
# TODO(dannas): Write solution for part 2

## [Day 8](http://adventofcode.com/2017/day/8): I Heard You Like Registers
Given a list of unusual register instructions, what is the largest value in any register after completing the instructions?

Each line contains a statement of two expressions. Let's start with parsing the lines. Note that array() returns ints or floats for numbers, not strings.

In [10]:
program = array(Input(8).read())
program[:5]

[['v', 'inc', 523.0, 'if', 't', '==', 6.0],
 ['qen', 'dec', -450.0, 'if', 'lht', '!=', 10.0],
 ['jyg', 'dec', -378.0, 'if', 'lb', '!=', -6.0],
 ['k', 'inc', -994.0, 'if', 'z', '<', 6.0],
 ['gjr', 'inc', -698.0, 'if', 'hbq', '<', 9.0]]

I started out with creating two functions; one for evaluating the conditional expression and one for the inc/dec expression. But I realized that they shared a lot of symmetry. Let's piggy-back on Pythons eval function.

In [13]:
def eval_expr(lhs, op, rhs):
    expr = '{} {} {}'.format(lhs, op, rhs)
    return eval(expr)

eval_expr(1, '<', 2)

True

I choose to represent the registers as a defaultdict, which will return 0 for unseen keys.

In [ ]:
def run(program):
    regs = defaultdict(int)
    for dst, op, imm1, _, src, cmp, imm2 in program:
        if eval_expr(regs[src], cmp, imm2):
            regs[dst] = eval_expr(regs[dst], '+' if op == 'inc' else '-', imm1)
    return max(regs.values())

run(program)

In **part 2** we're asked to find the highest value held in any register during this process.

The highest value will be returned from eval_expr. I used a decorator for storing the largest return value.

In [15]:
def record_max(f):
    record_max.maxval = -1
    def _f(*args):
        ret = f(*args)
        record_max.maxval = max(record_max.maxval, ret)
        return ret
    return _f

@record_max
def eval_expr(lhs, op, rhs):
    expr = '{} {} {}'.format(lhs, op, rhs)
    return eval(expr)

run(program)
print(record_max.maxval)

3818.0


## [Day 9](http://adventofcode.com/2017/day/9): Stream Processing
TODO(dannas): Write description

In [33]:
# TODO(dannas): Cleanup.
def tokens(stream):
    escape_seen = False
    #garbage_seen = False
    for t in stream:
        if escape_seen:
            escape_seen = False
            continue
        elif t == '!':
            escape_seen = True
            continue
        #elif not garbage_seen:
        #    if t == '<':
        #        garbage_seen = True
        #elif garbage_seen:
        #    if t == '>':
        #        garbage_seen = False
        #        continue
        #    else:
        #        continue
        yield t

def ignore_garbage(stream):
    in_garbage = False
    for t in tokens(stream):
        if in_garbage and t == '>':
            in_garbage = False
        elif not in_garbage and t == '<':
            in_garbage = True
        else:
            yield t
        
def scan_stream(stream):
    score = 0
    nesting = 0
    for t in ignore_garbage(stream):
        if t == '{':
            nesting += 1
        elif t == '}':
            score += nesting
            nesting -= 1
    return score

scan_stream(Input(9).read())
            
        
        

-330053

## [Day 10](http://adventofcode.com/2017/day/10): Knot Hash
TODO(dannas): Solve day 10

## [Day 11](http://adventofcode.com/2017/day/11): Hex Ed
TODO(dannas): Write description

In [13]:
directions = dict(s=(0, -2), sw=(-1, -1), se=(1, -1), n=(0, 2), nw=(-1, 1), ne=(1, 1))

def bfs(goal, startpos):
    visited = set()
    queue = deque()
    queue.append([startpos])
    while queue:
        steps = queue.popleft()
        for name, d in directions.items():
            newpos = move(steps[-1], d)
            if newpos not in visited:
                visited.add(newpos)
                newlist = steps + [newpos]
                if newpos == goal:
                    return newlist
                queue.append(newlist)
            
def walk(steps):
    pos = (0, 0)
    for d in steps:
        pos = move(pos, directions[d])
    return pos

#walk('ne,ne,ne'.split(','))
#walk('ne,ne,s,s'.split(','))
goal = walk(Input(11).read().strip().split(','))
print(goal)
path = bfs(goal, (0, 0))
len(path) - 1

(473, 891)


683

In **part 2** we're asked how many steps away he is as furthest from the starting position.

Now I can't rely on a brute force search. I'll have to figure out how to calculate the distance function for hex grids.

TODO(dannas): Solve part 2.

In [14]:
walk(Input(11).read().strip().split(','))

(473, 891)

In [21]:
pos = (0, 0)
print(X(pos))
while X(pos) < 473:
    pos = move(pos, directions['ne'])
print(pos)
print(891 - Y(pos))
print(473 + 418)

0
(473, 473)
418
891


In [3]:
def L(pos):
    return pos[0]
def U(pos):
    return pos[1]
def R(pos):
    return pos[2]

def move(pos, direction):
    return (L(pos) + L(direction), U(pos) + U(direction), R(pos) + R(direction))

def walk(steps):
    # left, up, right
    pos = (0, 0, 0)
    path = [pos]
    directions = dict(s=(0, -1, 0), sw=(0, -1, -1), se=(-1, -1,  0), n=(0, 1,  0), nw=(1, 1, 0), ne=(0, 1, 1))
    for d in steps:
        pos = move(pos, directions[d])
        path.append(pos)
    return path


path = walk(Input(11).read().strip().split(','))

def hexgrid_distance(p, q=(0, 0, 0)):
    diff = (L(p) - L(q), U(p) - U(q), R(p) - R(q))
    return max(mapt(abs, (diff)))

max(path, key=hexgrid_distance)


829

## [Day 12](http://adventofcode.com/2017/day/12): Digital Plumber
We are given a list of connections between "programs". Find out how many programs are reachable from the program with id 0?

This sounds like a good fit for the union-find algorithm. Start with identifying the largest id.

In [29]:
N = max(x for L in Input(12) for x in ints(line))
N

1999

Create an id array with N+1 elements. The value at each position represents the group id. So a[0] = 42 means that 0 belongs to group 42.

In [33]:
a = list(range(0, N+1))

def find(a, x):
    return a[x]

def union(a, x, y):
    px = find(a,x)
    py = find(a, y)
    if px ==  py:
        return
    for i in range(0, N+1):
        if a[i] == px:
            a[i] = py

Loop over pairs and call union on them.

In [34]:
def parse(line):
    x, neighbors = line.split('<->')
    for n in neighbors.split(', '):
        yield int(x), int(n)

for x, y in (pair for line in Input(12) for pair in parse(line)):
    union(a, x, y)

len([x for x in a if x == a[0]])

239

In **part 2** we're asked how many groups are there in total?

That would be the number of unique group ids.

In [31]:
len(set(a))

215